# Assigment 04 

In this Assigment, we are going to download XBRL files and extract some partes of the files.
We will create a funciontion to download the files and save into a especific folder.

The urls from the files are into a xml files, so we need to parse, find the right tags to get the links

In this assigment, insted of saving all files into memory, we will save into disk, extract only the parts we need than save into a dataframe.

In [1]:
#Importing dependencies
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc="Processing")

In [2]:
#This function will download the files and save into disk
def download_file(url, path='./'):
    import urllib.request
    import shutil
    
    file_name = path + url.split('/')[-1]

    # Download the file from `url` and save it locally under `file_name`:
    with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

In [3]:
#Path where files will saved
path ='./data/raw/xbrl/'

We can get the list of the files in https://www.sec.gov/structureddata/rss-feeds-submitted-filings

In [4]:
url = 'https://www.sec.gov/Archives/edgar/monthly/xbrlrss-2017-08.xml'

In [5]:
download_file(url,path=path)

Now we've downloaded the index files, let's parse and get the urls to download the xbrl files

In [6]:
#Creating variables to store the data.
companies = []
urls = []
file = []

#Open the xml file to parse and find the url for the xbrl files
xmlfile = path + url.split('/')[-1]
tree = ET.parse(xmlfile)
root = tree.getroot()
item = root.findall('channel/item')

#The for loop will parse the xml file to find the link to the xbrl files we need
for entry in item:   
    
    desc = entry.findtext('description')  
    
    if desc == '10-K':
        companyName = ''
        xbrlurl = ''
        for item in entry.iter():

            if item.tag == '{http://www.sec.gov/Archives/edgar}companyName':
                companyName = item.text
                
            if item.tag =='{http://www.sec.gov/Archives/edgar}xbrlFiles':
                for ite  in item:
                    if ite.attrib['{http://www.sec.gov/Archives/edgar}description'] == 'XBRL INSTANCE DOCUMENT':
                        xbrlurl = ite.attrib['{http://www.sec.gov/Archives/edgar}url']
        
        #Some companies dont have the xbrl file we are going to use
        if  xbrlurl != '': 
            companies.append(companyName)  
            urls.append(xbrlurl)
            file.append(xbrlurl.split('/')[-1])

In [7]:
#Creating a Dataframe to easely manipulate the data
data = pd.DataFrame(data ={'Company': companies,
                           'url': urls,
                           'file': file})

In [8]:
#Creating a sample of 100 documents
data = data.sample(100, random_state=42 )

In [9]:
download = data['url'].progress_apply(download_file, path= path)

Processing: 100%|██████████| 100/100 [01:07<00:00,  1.65it/s]


In [10]:
#Lets create a checkpoint for start here next time
data.to_json('./data/10k-xbrl.json')

# Parsing XBRL files

Now that we have all files, we need to find only the tags we need

In [11]:
def xbrl_tag(xbrlfile,path= './',tags=[]):
    import xml.etree.ElementTree as ET
    from bs4 import BeautifulSoup
    
    text = ''
    xbrlfile = path + xbrlfile
    tree = ET.parse(xbrlfile)
    root = tree.getroot()
    
    for item in root.findall('.//'):
        #Only will get the tag we need
        for tag in tags:
            if item.tag.find(tag) >= 0:          
                text = BeautifulSoup(item.text,"lxml-xml",).get_text()
    return text            

In [12]:
tags = ['CommitmentsAndContingenciesDisclosure', 'ContingenciesDisclosure']

In [13]:
data['Comitiment'] = data['file'].progress_apply(xbrl_tag,path=path,tags=tags)

Processing: 100%|██████████| 100/100 [00:07<00:00, 13.75it/s]


In [15]:
data[data['Comitiment'].apply(len) == 0].count()

Company       33
file          33
url           33
Comitiment    33
dtype: int64

we have 33 documents that don't have this tag, we need to figure out this.

I will update this notebook has soon as I finish